In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession \
    .builder \
    .getOrCreate()
sc = spark.sparkContext
df = spark.read.csv("/home/phani/Elasticsearch_spark/employee.csv", header=True)
df = df.fillna(0)
print(df.show())


+------+-----------+----------+--------------+---------+------+--------------------+----------------+------------------+--------------------+-------------+-------------+-----------+--------------+---------------+------------------+---------------+---------------+----------------+---------------------+-----------+--------------+--------------+---------+----------------------+------+-----------+-----------+------------+-------------+--------------------+-------------+-----+-----+---------+-----------+---------------+
|Emp ID|Name Prefix|First Name|Middle Initial|Last Name|Gender|              E Mail|   Father's Name|     Mother's Name|Mother's Maiden Name|Date of Birth|Time of Birth|Age in Yrs.|Weight in Kgs.|Date of Joining|Quarter of Joining|Half of Joining|Year of Joining|Month of Joining|Month Name of Joining|Short Month|Day of Joining|DOW of Joining|Short DOW|Age in Company (Years)|Salary|Last % Hike|        SSN|  Phone No. |   Place Name|              County|         City|State|  Z

In [3]:
print(df.printSchema())

root
 |-- Emp ID: string (nullable = true)
 |-- Name Prefix: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Middle Initial: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- E Mail: string (nullable = true)
 |-- Father's Name: string (nullable = true)
 |-- Mother's Name: string (nullable = true)
 |-- Mother's Maiden Name: string (nullable = true)
 |-- Date of Birth: string (nullable = true)
 |-- Time of Birth: string (nullable = true)
 |-- Age in Yrs.: string (nullable = true)
 |-- Weight in Kgs.: string (nullable = true)
 |-- Date of Joining: string (nullable = true)
 |-- Quarter of Joining: string (nullable = true)
 |-- Half of Joining: string (nullable = true)
 |-- Year of Joining: string (nullable = true)
 |-- Month of Joining: string (nullable = true)
 |-- Month Name of Joining: string (nullable = true)
 |-- Short Month: string (nullable = true)
 |-- Day of Joining: string (nullable = true)
 |-- DO

In [4]:
df1 = df.select('Emp ID', 'First Name', 'Middle Initial', 'Gender', 'E Mail', 'Short Month', 'Salary', 'County', 'City', 'State', 'Region', regexp_replace(col("Last % Hike"), "%", "").alias("Hike"))
df1.show()

+------+----------+--------------+------+--------------------+-----------+------+--------------------+-------------+-----+---------+----+
|Emp ID|First Name|Middle Initial|Gender|              E Mail|Short Month|Salary|              County|         City|State|   Region|Hike|
+------+----------+--------------+------+--------------------+-----------+------+--------------------+-------------+-----+---------+----+
|742048|    Lizeth|             P|     F|lizeth.mccoll@ibm...|        Jan|147446|               Stark|     Alliance|   OH|  Midwest|  14|
|671135| Argentina|             O|     F|argentina.hern@nt...|        Apr|129174|District of Columbia|   Washington|   DC|    South|   8|
|965851|    Damian|             N|     M|damian.patillo@ou...|        Dec|158746|              Fresno|       Burrel|   CA|     West|   8|
|224660|   Imogene|             P|     F|imogene.hagopian@...|        Jun| 55761|              Brazos|        Bryan|   TX|    South|  14|
|852694|    Walker|             E|

# Summarise the number of emp joined and hickes granted based on month

In [7]:
print("Total Number of Employees joined based on months")
df1.groupBy('Short Month').count().show()

Total Number of Employees joined based on months
+-----------+------+
|Short Month| count|
+-----------+------+
|        Oct|400048|
|        Sep|384646|
|        Dec|408721|
|        Aug|468295|
|        May|434522|
|        Jun|427424|
|        Feb|382263|
|        Nov|392382|
|        Mar|422987|
|        Jan|413247|
|        Apr|414484|
|        Jul|450981|
+-----------+------+



## Hikes granted based on Months

In [13]:
df1.groupBy("Short Month") \
    .agg(count("Hike").alias("Hikes"), \
        count("Emp ID").alias("Employees")).show()

+-----------+------+---------+
|Short Month| Hikes|Employees|
+-----------+------+---------+
|        Oct|400048|   400048|
|        Sep|384646|   384646|
|        Dec|408721|   408721|
|        Aug|468295|   468295|
|        May|434522|   434522|
|        Jun|427424|   427424|
|        Feb|382263|   382263|
|        Nov|392382|   392382|
|        Mar|422987|   422987|
|        Jan|413247|   413247|
|        Apr|414484|   414484|
|        Jul|450981|   450981|
+-----------+------+---------+



In [14]:
df1.groupBy("Short Month") \
    .agg(sum("Hike").alias("sum_Hike"), \
         avg("Hike").alias("avg_Hike"), \
         min("Hike").alias("min_Hike"), \
         max("Hike").alias("max_Hike") \
     ) \
    .show(truncate=False)


+-----------+---------+------------------+--------+--------+
|Short Month|sum_Hike |avg_Hike          |min_Hike|max_Hike|
+-----------+---------+------------------+--------+--------+
|Oct        |6005079.0|15.010896192456904|0       |9       |
|Sep        |5767464.0|14.994212860656292|0       |9       |
|Dec        |6131478.0|15.001622133435767|0       |9       |
|Aug        |7030207.0|15.012346918075146|0       |9       |
|May        |6509808.0|14.981538334077445|0       |9       |
|Jun        |6410015.0|14.996853241745901|0       |9       |
|Feb        |5734304.0|15.000939143992486|0       |9       |
|Nov        |5873249.0|14.968191711138635|0       |9       |
|Mar        |6353930.0|15.021572767011753|0       |9       |
|Jan        |6191321.0|14.982131751712656|0       |9       |
|Apr        |6218791.0|15.003693749336524|0       |9       |
|Jul        |6768162.0|15.007643337524197|0       |9       |
+-----------+---------+------------------+--------+--------+

